## Zadania: 5 zasad efektywnej inżynierii promptów
5 zasad efektywnej inżynierii promptów:
1. Jasne instrukcje poprawiają trafność.
2. Przykłady stabilizują odpowiedź.
3. Zdefiniowany format odpowiedzi = przewidywalny wynik.
4. Dzielenie na kroki = lepsze i pełniejsze rozwiązania.
5. Testy i weryfikacja = bezpieczeństwo i poprawność.

In [2]:
# Import bibliotek
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# model bazowy
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Zasada 1 — Jasne instrukcje

In [3]:
# ===============================
# - Zadanie: Napisz własny przykład złego i dobrego promptu Wypisz 2 różnice w otrzymanych rezultatach.

# Zły prompt — niejasny, bez roli i oczekiwań
bad_prompt = "Napisz funkcję w Pythonie."
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content)

# Dobry prompt — jasno określona rola i oczekiwania
good_prompt = """Jesteś ekspertem programującym w Pythonie.
Napisz funkcję w Pythonie, która przyjmuje listę liczb całkowitych
i zwraca nową listę zawierającą tylko liczby parzyste.
Dodaj test jednostkowy w pytest."""
print("\n=== Dobry prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, abym napisał? Możesz podać szczegóły dotyczące jej działania lub celu, a ja postaram się pomóc.

=== Dobry prompt ===
Oto przykładowa funkcja w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodatkowo dodam test jednostkowy z użyciem biblioteki `pytest`.

### Funkcja

```python
def get_even_numbers(numbers):
    """Funkcja zwraca listę liczb parzystych z podanej listy."""
    return [num for num in numbers if num % 2 == 0]
```

### Test jednostkowy

Aby przetestować funkcję, stworzymy plik testowy, na przykład `test_even_numbers.py`.

```python
import pytest
from your_module import get_even_numbers  # Upewnij się, że importujesz funkcję z odpowiedniego modułu

def test_get_even_numbers():
    assert get_even_numbers([1, 2, 3, 4, 5, 6]) == [2, 4, 6]
    assert get_even_numbers([10, 11, 12, 13, 14]) == [10, 12, 14]
    assert get_even_numbers([-2, -1, 0, 1, 2]) == [-2, 0, 2]
 

### Zasada 2 — Używaj przykładów

In [4]:
# ===============================
# Zadanie: wygeneruj tagi na podstawie treści strony firmy. Dodaj stopniowo kilka kolejnych przykładów opisu firmy i zweryfikuj jak zmieniają sie generowane wyniki wraz ze zwiększaniem licznb przykładów.

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto kilka tagów, które mogą opisać firmę Lego na podstawie podanego tekstu:

1. LEGO
2. zabawki
3. dzieci
4. producent zabawek
5. kreatywność
6. konstrukcyjne
7. edukacyjne
8. rozrywka
9. rozwój dzieci
10. marka globalna

=== One-shot ===
odzież, buty, sport

=== Few-shot ===
samochody, elektryczność, energia


### Zasada 3 — Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [5]:
# - Zadanie: Zmodyfikuj poniższy przykład tak, aby dotyczył generowania przepisu na twoje ulubione danie zamiast planu wycieczki. Dodaj reguły walidacji wejścia i wyjścia z wykorzystaniem biblioteki Pydantic.

from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Poznań", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Poznań' summary='Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.' must_do=['Odwiedź Stary Rynek i zobacz ratusz z koziołkami.', 'Spaceruj po Ostrówie Tumskim, historycznym centrum Poznania.', 'Zrelaksuj się w Parku Cytadela, gdzie znajdziesz muzea i piękne tereny zielone.', 'Spróbuj lokalnych specjałów w jednej z restauracji, np. pyry z gzikiem.', 'Zobacz Muzeum Narodowe z bogatą kolekcją sztuki.']
{
  "city": "Poznań",
  "summary": "Poznań to jedno z najstarszych miast w Polsce, znane z bogatej historii, pięknej architektury i tętniącej życiem atmosfery. To idealne miejsce na krótki wypad, oferujące zarówno zabytki, jak i nowoczesne atrakcje.",
  "must_do": [
    "Odwiedź Stary Rynek i zobacz ratusz z koziołkami.",
    "Spaceruj po Ostrówie Tumskim, historycznym centrum Poznania.",
    "Zrelaksuj się w Parku Cytadel

### Zasada 4 — Dziel złożone zadania na kroki

In [6]:
# ===============================
# - Zadanie: Zmodyfikuj poniższy przykład tak, aby w kilku krokach realizował zadanie: "Doradź, jakie auto powinienem/powinnam kupić?".

# Zły prompt — wszystko naraz
bad_prompt = """Przygotuj trzydniowy plan zwiedzania Poznania z budżetem 300 euro,
uwzględniając atrakcje, restauracje, transport i mapy."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Dobry prompt — krok po kroku
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz mapy.

### Dzień 1: Stare Miasto i okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (około 5 euro)
- **Atrakcja:** Stary Rynek, Ratusz (bezpłatnie)
- **Transport:** Spacer po Starym Mieście (bezpłatnie)

**Południe:**
- **Atrakcja:** Muzeum Narodowe (bilet wstępu około 5 euro)
- **Obiad:** Restauracja "Bamberka" (około 10 euro)

**Popołudnie:**
- **Atrakcja:** Ostrów Tumski, Katedra (bezpłatnie, ewentualnie 2 euro za wejście do katedry)
- **Transport:** Sp ...

=== Dobry prompt — krok 1 ===
Oto lista najważniejszych atrakcji kulturalnych w Poznaniu wraz z ich godzinami otwarcia. Proszę pamiętać, że godziny otwarcia mogą się zmieniać, dlatego zawsze warto sprawdzić aktualne informacje na oficjalnych stronach internetowych.

1. **Stary Rynek**
   - Opis: Serce Poznania, z pięknymi kamienicami i ratuszem.
   - Godziny otwarcia: Cało

### Zasada 5 — Testuj i weryfikuj wyniki

In [7]:
# ===============================
# - Zadanie: Zdefiniuj zakres ocen (skala 1–5)  - napisz dedydykowaną funkcję w Pythonie.
# - Zadanie: Oceń 2 odpowiedzi i policz średnią - napisz dedydykowaną funkcję w Pythonie.

from langchain.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Warszawa jest stolicą Polski"
)

print(result)


{'score': 0.055613485077632974}
